<a href="https://colab.research.google.com/github/samrat9474457-a11y/Suraksha_AI/blob/main/Suraksha_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Suraksha_Ai**

**Importing libraries**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

**loading the dataset**


In [4]:
df = pd.read_csv('/content/crime_in_la.csv', on_bad_lines='skip', engine='python')
df

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,LOCATION,LAT,LON
0,190326475,01/03/2020,21:30,7,Wilshire,784,510,VEHICLE - STOLEN,0,0.0,M,O,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,AA,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506
1,200106753,08/02/2020,18:00,1,Central,182,330,BURGLARY FROM VEHICLE,1822 1402 0344,47.0,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628
2,200320258,04/11/2020,17:00,3,Southwest,356,480,BIKE - STOLEN,0344 1251,19.0,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1400 W 37TH ST,34.0210,-118.3002
3,200907217,10/03/2020,20:37,9,Van Nuys,964,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19.0,M,O,CLOTHING STORE,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387
4,220614831,17/08/2020,12:00,6,Hollywood,666,354,THEFT OF IDENTITY,1822 1501 0930 2004,28.0,M,H,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1900 TRANSIENT,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,201821588,03/12/2020,16:00,18,Southeast,1823,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329 1822 1300 1601 0342 2024,23.0,F,H,"VEHICLE, PASSENGER/TRUCK",500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,103RD ST,33.9429,-118.2652
4616,201517594,15/11/2020,20:30,15,N Hollywood,1557,624,BATTERY - SIMPLE ASSAULT,0400,23.0,F,H,RESTAURANT/FAST FOOD,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,10900 MAGNOLIA BL,34.1649,-118.3686
4617,201413601,10/07/2020,19:20,14,Pacific,1411,440,THEFT PLAIN - PETTY ($950 & UNDER),2004 0344 1218,41.0,F,O,PARKING LOT,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,OCEAN FRONT WK,33.9940,-118.4799
4618,201305733,27/01/2020,09:00,13,Newton,1324,624,BATTERY - SIMPLE ASSAULT,1202 0446,60.0,M,H,MARKET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,2000 S CENTRAL AV,34.0233,-118.2516


**data processing**

In [5]:
df.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc',
       'Status', 'Status Desc', 'LOCATION', 'LAT', 'LON'],
      dtype='object')

In [6]:
 df = df[['LAT', 'LON', 'TIME OCC', 'Crm Cd Desc']]
df = df.dropna()

In [7]:
df['TIME OCC'] = pd.to_numeric(df['TIME OCC'], errors='coerce')
df['HOUR'] = df['TIME OCC'] // 100
df.drop(columns=['TIME OCC'], inplace=True)

**fixing the thresold**

In [8]:
def danger_level(crime):
    crime = crime.lower()
    if 'rape' in crime or 'homicide' in crime or 'assault with deadly weapon' in crime or 'kidnap' in crime:
        return 2   # Dangerous
    elif 'robbery' in crime or 'assault' in crime or 'sexual' in crime:
        return 1   # Not Safe
    else:
        return 0   # Safe

df['danger'] = df['Crm Cd Desc'].apply(danger_level)
df.drop(columns=['Crm Cd Desc'], inplace=True)

In [9]:
X = df[['LAT', 'LON', 'HOUR']]
y = df['danger']

**splitting the dataset into train and test**

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

**importing the pre trained model**


In [11]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**testing the accuracy**

In [12]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)


0.7207792207792207

In [13]:
# Example: Night time location
sample = [[34.0407, -118.2468, 19]]  # LAT, LON, HOUR
prediction = model.predict(sample)

labels = ['Safe', 'Not Safe', 'Dangerous']
labels[prediction[0]]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Not Safe'